In [13]:
import pickle
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

In [4]:
s_data = pickle.load(open("sensitivity.p", "rb"))

In [6]:
s_data.keys()

dict_keys(['p0', 'b0', 'c0', 's0', 'dels+', 'delb+', 'dels-', 'delc-', 'delb-', 'pi0', 'ap', 'lambda', 'u0', 'd0', 'ec0', 'es0', 'r0'])

In [38]:
keys = list(s_data.keys())

In [25]:
s_data["p0"][0.5][10]["stock"]["buildings"]

736.8809025897953

In [34]:
building_10 = {}
building_10["low"] = []
building_10["original"] = []
building_10["high"] = []
for k in keys:
    building_10["low"].append(s_data[k][0.5][10]["stock"]["buildings"] )
    building_10["original"].append(s_data[k][1][10]["stock"]["buildings"])
    building_10["high"].append(s_data[k][1.5][10]["stock"]["buildings"])
    

In [45]:
# only look at differences 
b10 = {}
b10["low"] = [np.abs(i - j) for i, j in zip(building_10["low"], building_10["original"])]

b10["high"] = [np.abs(i - j) for i, j in zip(building_10["low"], building_10["high"])]

b10["low"]  = [-1*i for i in b10["low"]]

In [50]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y = keys,
    x = b10["low"],
    name = "Low",
    orientation="h",
    marker_color = "rgb( 158, 202, 225)"
))

fig.add_trace(go.Bar(
    y = keys,
    x = b10["high"],
    name = "High",
    orientation="h",
    marker_color = "Crimson"
))

fig.update_layout(barmode="overlay")

fig.show()